<a href="https://colab.research.google.com/github/adigenova/uohpmd/blob/main/code/Arango_GraphDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ArangoDB Trabajando con grafos

ArangoDB es una base de datos multimodelo  con modelos de datos flexibles para documentos, grafos y clave/valor. Posee el lenguaje AQL que es similar al SQL para realizar consultas.

# Configuración

Antes de comenzar con ArangoDB, debemos preparar nuestro entorno y crear una base de datos temporal en el Servicio Oasis.

In [1]:
%%capture
!git clone https://github.com/joerg84/ArangoDBUniversity.git
!rsync -av ArangoDBUniversity/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [2]:
import json
import requests
import sys
import oasis
import time
from IPython.display import JSON

from arango import ArangoClient

Creamos una base de datos temporal

In [3]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="Pregel", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
db = oasis.connect_python_arango(login)

Requesting new temp credentials.
Temp database ready to use.


In [4]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

https://tutorials.arangodb.cloud:8529
Username: TUTqgv02hwotei9n8jzqat7l
Password: TUTbnn6oxil354wmda8zwt9s
Database: TUTxry648yhncmgknxhbv8xkt


Podemos usar la URL anterior para revisar nuestra base de datos arango temporal.



# Cargando datos en arangoDB

In [5]:
! zcat /content/aeropuertos.csv.gz | wc -l
! zcat /content/vuelos.csv.gz | wc -l 
! zcat /content/aeropuertos.csv.gz > /content/aeropuertos.csv
! zcat /content/vuelos.csv.gz > /content/vuelos.csv
! echo ":::AEROPUERTOS:::"
! head /content/aeropuertos.csv
! echo ":::VUELOS:::"
! head /content/vuelos.csv

3376
286464
:::AEROPUERTOS:::
"_key","name","city","state","country","lat","long","vip"
"00M","Thigpen ","Bay Springs","MS","USA",31.95376472,-89.23450472,false
"00R","Livingston Municipal","Livingston","TX","USA",30.68586111,-95.01792778,false
"00V","Meadow Lake","Colorado Springs","CO","USA",38.94574889,-104.5698933,false
"01G","Perry-Warsaw","Perry","NY","USA",42.74134667,-78.05208056,false
"01J","Hilliard Airpark","Hilliard","FL","USA",30.6880125,-81.90594389,false
"01M","Tishomingo County","Belmont","MS","USA",34.49166667,-88.20111111,false
"02A","Gragg-Wade ","Clanton","AL","USA",32.85048667,-86.61145333,false
"02C","Capitol","Brookfield","WI","USA",43.08751,-88.17786917,false
"02G","Columbiana County","East Liverpool","OH","USA",40.67331278,-80.64140639,false
:::VUELOS:::
"_from","_to","Year","Month","Day","DayOfWeek","DepTime","ArrTime","DepTimeUTC","ArrTimeUTC","UniqueCarrier","FlightNum","TailNum","Distance"
"airports/ATL","airports/CHS",2008,1,1,2,2,57,"2008-01-01T05:02:00.0

In [6]:
# cargamos los aeropuertos A LA db
! ./tools/arangoimport  -c none --server.endpoint http+ssl://{login["hostname"]}:{login["port"]} --server.username {login["username"]} --server.database {login["dbName"]} --server.password {login["password"]} --file /content/aeropuertos.csv --collection airports --create-collection true --type csv

Connected to ArangoDB 'http+ssl://tutorials.arangodb.cloud:8529, database: 'TUTxry648yhncmgknxhbv8xkt', username: 'TUTqgv02hwotei9n8jzqat7l'
----------------------------------------
database:               TUTxry648yhncmgknxhbv8xkt
collection:             airports
create:                 yes
create database:        no
source filename:        /content/aeropuertos.csv
file type:              csv
quote:                  "
separator:              
threads:                2
connect timeout:        5
request timeout:        1200
----------------------------------------
Starting CSV import...
2022-11-07T14:37:35Z [179] INFO [9ddf3] processed 32768 bytes (3%) of input file
2022-11-07T14:37:35Z [179] INFO [9ddf3] processed 65536 bytes (15%) of input file
2022-11-07T14:37:35Z [179] INFO [9ddf3] processed 98304 bytes (27%) of input file
2022-11-07T14:37:35Z [179] INFO [9ddf3] processed 131072 bytes (40%) of input file
2022-11-07T14:37:35Z [179] INFO [9ddf3] processed 163840 bytes (52%) of input f

In [7]:
# CARGAMOS los vuelos a la DB
! ./tools/arangoimport  -c none --server.endpoint http+ssl://{login["hostname"]}:{login["port"]} --server.username {login["username"]} --server.database {login["dbName"]} --server.password {login["password"]} --file /content/vuelos.csv --collection flights --create-collection true --type csv --create-collection-type edge

Connected to ArangoDB 'http+ssl://tutorials.arangodb.cloud:8529, database: 'TUTxry648yhncmgknxhbv8xkt', username: 'TUTqgv02hwotei9n8jzqat7l'
----------------------------------------
database:               TUTxry648yhncmgknxhbv8xkt
collection:             flights
create:                 yes
create database:        no
source filename:        /content/vuelos.csv
file type:              csv
quote:                  "
separator:              
threads:                2
connect timeout:        5
request timeout:        1200
----------------------------------------
Starting CSV import...
2022-11-07T14:37:51Z [189] INFO [9ddf3] processed 1114112 bytes (3%) of input file
2022-11-07T14:37:51Z [189] INFO [9ddf3] processed 2195456 bytes (6%) of input file
2022-11-07T14:37:53Z [189] INFO [9ddf3] processed 3309568 bytes (9%) of input file
2022-11-07T14:37:53Z [189] INFO [9ddf3] processed 4390912 bytes (12%) of input file
2022-11-07T14:37:55Z [189] INFO [9ddf3] processed 5505024 bytes (15%) of input f

In [8]:
aql = db.aql

In [24]:
#Determine los nombres de todos los aeropuertos a los que se puede llegar directamente (1 paso) 
# desde LAX.
query = """
WITH airports
FOR a IN 1..1 OUTBOUND 'airports/LAX' flights
RETURN DISTINCT [a._key,a.name,a.city]
"""
query_result = aql.execute(query) 
for doc in  query_result:
 print(doc)

['JFK', 'John F Kennedy Intl', 'New York']
['SNA', 'John Wayne /Orange Co', 'Santa Ana']
['DEN', 'Denver Intl', 'Denver']
['CVG', 'Cincinnati Northern Kentucky Intl', 'Covington']
['SMF', 'Sacramento International', 'Sacramento']
['ATL', 'William B Hartsfield-Atlanta Intl', 'Atlanta']
['IYK', 'Inyokern', 'Inyokern']
['MSP', 'Minneapolis-St Paul Intl', 'Minneapolis']
['DFW', 'Dallas-Fort Worth International', 'Dallas-Fort Worth']
['PHL', 'Philadelphia Intl', 'Philadelphia']
['IAH', 'George Bush Intercontinental', 'Houston']
['TUS', 'Tucson International', 'Tucson']
['IPL', 'Imperial County', 'Imperial']
['ORD', "Chicago O'Hare International", 'Chicago']
['SBA', 'Santa Barbara Municipal', 'Santa Barbara']
['SAN', 'San Diego International-Lindbergh ', 'San Diego']
['FAT', 'Fresno Yosemite International', 'Fresno']
['SJC', 'San Jose International', 'San Jose']
['SEA', 'Seattle-Tacoma Intl', 'Seattle']
['OAK', 'Metropolitan Oakland International', 'Oakland']
['LAS', 'McCarran International'

In [ ]:
# Listar todos l
query = """
WITH airports
FOR airport, flight IN ANY 'airports/LAX' flights
FILTER flight.Month == 1 AND flight.Day >= 5 AND flight.Day <= 7
RETURN { city: airport.city, time: flight.ArrTimeUTC }
"""
query_result = aql.execute(query) 
for doc in  query_result:
 print(doc)

In [ ]:
# Listar todos los vuelos del avion N238JB
query = """
FOR flight IN flights
FILTER flight.TailNum == "N238JB" 
RETURN flight
"""
query_result = aql.execute(query) 
for doc in  query_result:
 print(doc)
 

In [20]:
# Determinar todos los aeropuertos directamente accesibles desde LAX
query = """
WITH airports
FOR airport IN OUTBOUND 'airports/LAX' flights 
 OPTIONS { bfs: true, uniqueVertices: 'global' } 
  RETURN airport
"""
query_result = aql.execute(query) 
for doc in  query_result:
 print(doc)


{'_key': 'JFK', '_id': 'airports/JFK', '_rev': '_fDIEcqS--0', 'name': 'John F Kennedy Intl', 'city': 'New York', 'state': 'NY', 'country': 'USA', 'lat': 40.63975111, 'long': -73.77892556, 'vip': True}
{'_key': 'SNA', '_id': 'airports/SNA', '_rev': '_fDIEcrG-_A', 'name': 'John Wayne /Orange Co', 'city': 'Santa Ana', 'state': 'CA', 'country': 'USA', 'lat': 33.67565861, 'long': -117.8682225, 'vip': False}
{'_key': 'DEN', '_id': 'airports/DEN', '_rev': '_fDIEcpy-_O', 'name': 'Denver Intl', 'city': 'Denver', 'state': 'CO', 'country': 'USA', 'lat': 39.85840806, 'long': -104.6670019, 'vip': False}
{'_key': 'CVG', '_id': 'airports/CVG', '_rev': '_fDIEcpy--S', 'name': 'Cincinnati Northern Kentucky Intl', 'city': 'Covington', 'state': 'KY', 'country': 'USA', 'lat': 39.04614278, 'long': -84.6621725, 'vip': False}
{'_key': 'SMF', '_id': 'airports/SMF', '_rev': '_fDIEcrG--6', 'name': 'Sacramento International', 'city': 'Sacramento', 'state': 'CA', 'country': 'USA', 'lat': 38.69542167, 'long': -121.

In [22]:
# Encuentre la ruta más corta entre el aeropuerto BIS y el aeropuerto JFK y 
# devuelva los nombres de los aeropuertos en la ruta.

query = """
WITH airports
FOR v IN OUTBOUND SHORTEST_PATH 'airports/BIS' TO 'airports/JFK' flights RETURN v.name
"""
query_result = aql.execute(query) 
for doc in  query_result:
 print(doc)


Bismarck Municipal
Denver Intl
John F Kennedy Intl
